## Step 1: Split up images into 256x256 images

In [33]:
import os

def rename_files(input_folder):
    # List all files in the directory
    files = os.listdir(input_folder)

    for file in files:
        # Check if the filename starts with 'mask_'
        if file.startswith("mask_"):
            # Generate the new filename by removing 'mask_'
            new_filename = file.replace("mask_", "")
            # Create full paths
            original_path = os.path.join(input_folder, file)
            new_path = os.path.join(input_folder, new_filename)
            # Rename the file
            os.rename(original_path, new_path)
            print(f"Renamed '{file}' to '{new_filename}'")

if __name__ == "__main__":
    # Specify the folder containing the mask files
    input_folder = "./temp/train_masks"  # Change this to your actual folder path
    rename_files(input_folder)

In [34]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify

original_height = None
original_width = None

def create_mask(input_folder, output_folder):
    global original_height, original_width
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of image filenames in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        # Read the image
        image_path = os.path.join(input_folder, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Unable to open image file {image_path}")
            continue
            
#         original_height, original_width = image.shape[:2]
        if original_height is None or original_width is None:
                    original_height, original_width = image.shape[:2]

        # Resize the image to make dimensions divisible by 256
        height, width = image.shape[:2]
        new_height = height - (height % 256)
        new_width = width - (width % 256)

        if new_height == 0 or new_width == 0:
            print(f"Image {image_path} is too small for resizing.")
            continue

        image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Create patches
        patches_img = patchify(image, (256, 256, 3), step=256)
        for i in range(patches_img.shape[0]):
            for j in range(patches_img.shape[1]):
                single_patch_img = patches_img[i, j, 0]
                # Save the patch
                mask_filename = os.path.join(output_folder, f"{image_file[:-4]}_{i}_{j}.jpg")
                cv2.imwrite(mask_filename, single_patch_img)

    print("Masks created and saved.")

if __name__ == "__main__":
    # Input folder containing original images
    images_input = "./temp/train_images"
    
    # Output folder for images
    images_output = "./temp/p_train_images"

    # Create masks
    create_mask(images_input, images_output)
    
    # Input folder containing annotated images
    masks_input = "./temp/train_masks"
    
    # Output folder for masks
    masks_output = "./temp/p_train_masks"

    # Create masks
    create_mask(masks_input, masks_output)


Masks created and saved.
Masks created and saved.


## Step 2: Run Prediction for Each 256x256 image

In [35]:
# util.py
# load packet
# check accuracy
# model.eval() predict



## Step 3: Reconstruct the Original Image and Mask

In [36]:
def reconstruct_image(patches_folder, reconstructed_folder):
    global original_height, original_width
    # Create the reconstructed folder if it doesn't exist
    if not os.path.exists(reconstructed_folder):
        os.makedirs(reconstructed_folder)

    files = os.listdir(patches_folder)
    unique_ids = set('_'.join(file.split('_')[:2]) for file in files if file.endswith(('.jpg', '.jpeg', '.png')))

    for image_id in unique_ids:
        patch_files = [f for f in files if f.startswith(image_id) and f.endswith(('.jpg', '.jpeg', '.png'))]
        patch_files.sort()

        # Determine dimensions for reconstruction
        max_i = max_j = 0
        for file in patch_files:
            parts = file.split('_')
            i, j = int(parts[2]), int(parts[3].split('.')[0])
            max_i = max(max_i, i)
            max_j = max(max_j, j)

        patch_size = 256
        height = (max_i + 1) * patch_size
        width = (max_j + 1) * patch_size
        reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

        # Place each patch in the correct position
        for file in patch_files:
            parts = file.split('_')
            i, j = int(parts[2]), int(parts[3].split('.')[0])
            patch = cv2.imread(os.path.join(patches_folder, file))
            reconstructed_image[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] = patch

        # Save the reconstructed image
        reconstructed_filename = os.path.join(reconstructed_folder, f'{image_id}.jpg')
        cv2.imwrite(reconstructed_filename, reconstructed_image)
        
        # Resize to the original dimensions
        resized_image = cv2.resize(reconstructed_image, (original_width, original_height), interpolation=cv2.INTER_AREA)
        reconstructed_filename = os.path.join(reconstructed_folder, f'{image_id}.jpg')
        cv2.imwrite(reconstructed_filename, resized_image)

    print("Images reconstructed and saved.")

if __name__ == "__main__":
    
    # IMAGES ######################################################
    # Paths for input and output
    input_folder = "./temp/train_images"
    output_folder = "./temp/p_train_images"
    reconstructed_folder = "./temp/reconstructed_images/images"

    # Reconstruct images
    reconstruct_image(output_folder, reconstructed_folder)
    
    # MASKS #######################################################
    input_folder = "./temp/train_masks"
    output_folder = "./temp/p_train_masks"
    reconstructed_folder = "./temp/reconstructed_images/masks"

    # Reconstruct images
    reconstruct_image(output_folder, reconstructed_folder)

Images reconstructed and saved.
Images reconstructed and saved.
